# Code Assigment 1

For this assignment you will use the following SVM implementation for classifying these datasets:
https://archive.ics.uci.edu/ml/datasets/banknote+authentication


https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

You should:

1) Specify which Machine Learning problem are you solving.

2) Provide a short summary of the features and the labels you are working on.

3) Please answer the following questions: a) Are these datasets linearly separable? b) Are these datasets randomly chosen and c) The sample size is enough to guarantee generalization.

4) Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

5) Show some examples to illustrate that the method is working properly.

6) Provide quantitative evidence for generalization using the provided dataset.


--- 

A continuación se van ha resolver dos problemas de clasificación binaria:

- **Clasificación de billetes**: dado un vector de características de la imagen de un billete se quiere determinar si el billete es falso o no. Para este problema se aplicó la transformada Wavelet a la imagen ,esta transformada divide la imagen en distintas frecuencias y me retorna una serie de coeficientes, coeficientes de detalle y de aproximación.El dataset tiene un vector de característica conformado por lo siguiente:

    - variance of Wavelet Transformed image : dispersion de los coeficientes de la transformada.
    - skewness of Wavelet Transformed image : indica la asimetría de la distribución de los coeficientes de la transformada,en este contexto nos puede dar información acerca de los bordes de la imagen.
    - curtosis of Wavelet Transformed image : caracteriza la elevación o achatamiento de la distribucion de los coeficientes de la transformada, en este contexto puede caracterizar la textura o rugosidad de la imagen.
    - entropy of image : nos dice acerca de la cantidad de información de la imagen,si es cercano a cero la imagen puede ser uniforme o suave, por el contrario entre mayor sea la entropia la imagen tiene más detalles.
    
    por último el dataset cuenta con una etiqueta binaria que nos indica el valor real de clasificación de la imagen, 0 si es falso el billete y 1 si el billete es verdadero.


- **Clasificación ocupación de una oficina**: dado un vector de características de las condiciones de una oficina se quiere determinar si la oficina está ocupada o no. El dataset cuenta con el siguiente vector de características:

    - date time year-month-day hour: instante en el cual se tomá la medición.
    - Temperature, in Celsius: temperatura de la oficina.
    - Relative Humidity, %: cantidad de vapor de agua presente en el aire en comparación con la cantidad máxima que el aire puede contener a una temperatura y presión determinadas.
    - Light, in Lux: cantidad de luz visible que llega a una superficie.
    - CO2, in ppm: concentración de CO2.
    - Humidity Ratio in kgwater-vapor/kg-air: cantidad de vapor de agua presente en el aire.

    por último el dataset cuenta con una etiqueta binaria que nos indica si la oficina está vacía o no, 0 si está vacía y 1 si está ocupada.

Se usará SVM como herramienta para solucionar el problema de clasificación, en particular se usará el planteamiento soft-margin usando la Hinge loss function y para optimizar el hiperplano se usará gradiente de descenso. A continuación se presenta el código del que se hace uso para resolver los problemas.

In [39]:
#Import neccesary libraries to run the code
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [40]:
#reference from: https://towardsdatascience.com/implementing-svm-from-scratch-784e4ad0bc6a

class SVM:
    def __init__(self, learning_rate=1e-3, n_iters=1000):
        # initialize the hyperparameters
        self.lr = learning_rate
        self.n_iters = n_iters
        
        # initialize the model parameters
        self.w = None
        self.b = None

    def _init_weights_bias(self, X):
        # initialize the weight vector and bias term to zero
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.b = 0

    def _get_cls_map(self, y):
        # convert the binary labels to a classification map
        return np.where(y <= 0, -1, 1)

    def _satisfy_constraint(self, x, idx):
        # check whether a training example satisfies the margin constraint
        linear_model = np.dot(x, self.w) + self.b 
        return self.cls_map[idx] * linear_model >= 1
    
    def _get_gradients(self, constrain, x, idx):
        # calculate the gradients of the loss function with respect to the model parameters
        if constrain:
            # if a point satisfies the margin constraint, only the regularization term is used in the gradient calculation
            dw = self.w
            db = 0
            return dw, db
        
        # if a point does not satisfy the margin constraint, both the regularization and hinge loss terms are used in the gradient calculation
        dw = self.w - np.dot(self.cls_map[idx], x)
        db = - self.cls_map[idx]
        return dw, db
    
    def _update_weights_bias(self, dw, db):
        # update the model parameters using the gradients and the learning rate
        self.w -= self.lr * dw
        self.b -= self.lr * db
    
    def fit(self, X, y):
        # train the SVM model using gradient descent
        self._init_weights_bias(X)
        self.cls_map = self._get_cls_map(y)

        for _ in range(self.n_iters):
            for idx, x in enumerate(X):
                # check whether the current training example satisfies the margin constraint
                constrain = self._satisfy_constraint(x, idx)
                
                # calculate the gradients for the current training example
                dw, db = self._get_gradients(constrain, x, idx)
                
                # update the model parameters using the gradients and the learning rate
                self._update_weights_bias(dw, db)
    
    def predict(self, X):
        # predict the class labels for new examples
        estimate = np.dot(X, self.w) + self.b
        prediction = np.sign(estimate)
        return np.where(prediction == -1, 0, 1)


In [41]:
#function to calculate accuracy
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true==y_pred) / len(y_true)
    return accuracy

para verificar si los datos son linealmente separables se hará uso de la librería sklearn la cual implementa svm y la cual nos retorna las componentes del vector $w$ y el intercepto $b$ que forman el hiperplano que separa los datos en caso de que exista, caso contrario devolveria coeficientes nulos.

In [42]:
from sklearn import svm
import numpy as np

# Function to check if tho sets of vectors are linear separables

def isLinear(X,Y):
    # Create a SVM classifier object with a linear kernel
    clf = svm.SVC(kernel='linear')
    # Train the classifier on the data
    clf.fit(X.values, Y.values)

    # Comprobar si el clasificador aprendió un hiperplano de separación válido
    if clf.coef_ is not None:
        # Check if the classifier learned a valid separating hyperplane
        print("Intercept:", clf.intercept_)
        print("Coefficients:", clf.coef_)
        print("Los datos son linealmente separables.")
    else:
        print("Los datos no son linealmente separables.")

### Clasificación de billetes

Como método para validar la generalización del modelo, y dado que no se poseen más datos, se realizará una partición del dataset original en proporcion 80-20, 80% de los datos se usarán para entrenar el modelo y el otro 20% para ver el comportamiento del modelo con nuevos datos.

In [43]:
#import the data
columns_dataset=["Variance W.T","Skewness W.T","Curtosis of W.T","Entropy Image","Class"]
df=pd.read_csv('data/bank_note/data_banknote_authentication.txt',names=columns_dataset)

#Set of features vectors
X=df[["Variance W.T","Skewness W.T","Curtosis of W.T","Entropy Image"]]
#Labels
y=df["Class"]

#Split the data in train and test with a test size of 0.2
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


a continuación se cuenta el número de muestras que se tiene por cada grupo, obteniendo que hay un número de muestras similar por grupo y de esta manera la muestra en principio representa a cada grupo bajo el supuesto de que la muestra total fue aleatoria y es representativa.

In [44]:
df["Class"].value_counts()

0    762
1    610
Name: Class, dtype: int64

In [45]:
#create instance of svm.
banknote = SVM() 

#fit the model with the train data
banknote.fit(x_train.values,y_train.values) 

In [46]:
#make predictions for the test data
predictions = banknote.predict(x_test.values)

#Accuracy of the model
print("SVM Accuracy: ", accuracy(y_test.values, predictions))

SVM Accuracy:  0.9854545454545455


De acuerdo a lo anterior, se obtuvo que el modelo logró clasificar correctamente el 98.5% de los nuevos datos que tuvo como entrada, lo que nos da indicios de que en principio logró una buena generalización. De acuerdo a esto el número de datos fue suficiente para lograr generalizar, bajo el supuesto de que la muestra es representativa de todos los billetes falsos y reales.

Ahora realizaremos la prueba con SVM de sklearn sobre el dataset completo para verificar que en realidad los grupos de datos son linealmente separables.

In [47]:
isLinear(X,y)

Intercept: [2.3989271]
Coefficients: [[-2.49568987 -1.44322744 -1.73193881 -0.25120478]]
Los datos son linealmente separables.


Por último a continuación se muestran algunos ejemplos del funcionamiento del modelo, tomando algunos ejemplos aleatorios del dataset de test

In [48]:
sample=df.sample()
feature_sample=sample[["Variance W.T","Skewness W.T","Curtosis of W.T","Entropy Image"]]
real_value=sample["Class"]
prediction_value=banknote.predict(feature_sample.values)

print(f"el valor real es: {real_value.iloc[0]} \nel valor de la predicción es: {prediction_value[0].item()}")

el valor real es: 0 
el valor de la predicción es: 0


--- 

### Ocupación de oficina

En este caso se cuenta con un dataset de entrenamiento y dos datasets de prueba para ver el rendimiento del modelo con datos no vistos con anterioridad.Dado que el atributo $date$ es un string, se decide tomar año,mes,dia,hora y minutos como atributos por separado.

In [49]:
#import the data
columns_dataset2=["date","Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy"]
train=pd.read_csv('data/occupancy_data/datatraining.txt',names=columns_dataset2,header=0)
test1=pd.read_csv('data/occupancy_data/datatest.txt',names=columns_dataset2,header=0)
test2=pd.read_csv('data/occupancy_data/datatest2.txt',names=columns_dataset2,header=0)


In [50]:
#TRAIN
#Separate the feature of date in day,month,year,hour,minute
train["date"]=pd.to_datetime(train["date"])
train['day'] = train["date"].dt.day
train['month'] = train["date"].dt.month
train['year'] = train["date"].dt.year
train['hour'] = train["date"].dt.hour
train['minute'] = train["date"].dt.minute
#Set of features vectors 
x_train2=train[["Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy","day","month","year","hour","minute"]]
#Labels
y_train2=train["Occupancy"]

In [51]:
#TEST 1
#Separate the feature of date in day,month,year,hour,minute
test1["date"]=pd.to_datetime(test1["date"])
test1['day'] = test1["date"].dt.day
test1['month'] = test1["date"].dt.month
test1['year'] = test1["date"].dt.year
test1['hour'] = test1["date"].dt.hour
test1['minute'] = test1["date"].dt.minute
#Set of features vectors 
x_test1=test1[["Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy","day","month","year","hour","minute"]]
#Labels
y_test1=test1["Occupancy"]

In [52]:
#TEST 2
#Separate the feature of date in day,month,year,hour,minute
test2["date"]=pd.to_datetime(test2["date"])
test2['day'] = test2["date"].dt.day
test2['month'] = test2["date"].dt.month
test2['year'] = test2["date"].dt.year
test2['hour'] = test2["date"].dt.hour
test2['minute'] = test2["date"].dt.minute
#Set of features vectors 
x_test2=test2[["Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy","day","month","year","hour","minute"]]
#Labels
y_test2=test2["Occupancy"]

A continuación se muestra la distribución de cada una de las categorías por cada conjunto de datos, con lo que se obtiene que el número de muestras correspondientes a la categoría de la oficina vacía en el dataset de entrenamiento y el segundo dataset de test corresponden alrededor del 78% de la muestra,mientras que en el primer dataset de test corresponde al 63% de la muestra, por lo que en principio sugiere que las muestras del primer dataset de test corresponden a un intervalo de tiempo con un comportamiento distinto en la oficina. Por otra parte teniendo en cuenta que cada dataset corresponde a cierto intervalo de tiempo, el comportamiento en cada intervalo puede variar drasticamente, por lo que tomar solo ciertos intervalos como muestra puede no llegar a representar el comportamiento de la ocupación de la oficina en el día. Lo ideal sería tomar aleatoriamente muestras durante todo el día, toda la semana, todo el mes, y todo el año para lograr una mejor generalización.

In [53]:
print("Muestra para el conjunto de entrenamiento\n",train["Occupancy"].value_counts(),"\n")
print("Muestra para el primer conjunto de test\n",test1["Occupancy"].value_counts(),"\n")
print("Muestra para el segundo conjunto de test\n",test2["Occupancy"].value_counts(),"\n")

Muestra para el conjunto de entrenamiento
 0    6414
1    1729
Name: Occupancy, dtype: int64 

Muestra para el primer conjunto de test
 0    1693
1     972
Name: Occupancy, dtype: int64 

Muestra para el segundo conjunto de test
 0    7703
1    2049
Name: Occupancy, dtype: int64 



A continuación entrenamos nuestro modelo, y a diferencía del punto anterior, se decidió usar un learning_rate mas pequeño con el fin de que pueda encontrar una mejor solución.

In [54]:
#create instance of svm
office_ocupancy = SVM(learning_rate=0.00001) 

#fit the model with the train data
office_ocupancy.fit(x_train2.values,y_train2.values) 

In [55]:
#make predictions for the test 1 data
predictions = office_ocupancy.predict(x_test1.values)

#Accuracy of the model
print("SVM Accuracy: ", accuracy(y_test1.values, predictions))

SVM Accuracy:  0.8881801125703565


Para el primer dataset de prueba se obtuvo una precisión del 88%, con lo cual para los intervalos de tiempo en los que se encuentran las muestras logra tener una buena generalización, sin embargo esto no implíca que haya generalizado por completo, faltarían más datos en intervalos de tiempo distinto o epocas del año distinto en donde las condiciones pueden variar y ahí si observar el rendimiento del modelo. 

In [56]:
#make predictions for the test 2 data
predictions = office_ocupancy.predict(x_train2.values)

#Accuracy of the model
print("SVM Accuracy: ", accuracy(y_train2.values, predictions))

SVM Accuracy:  0.8587744074665357


Para el segundo dataset de prueba se obtuvo una precisión del 86%, similar al anterior dataset de entrenamiento, habría que tener más datos para determinar mejor la generalización del modelo. Por otra parte el rendimiento del modelo puede que se deba a que la muestra se tomó sobre cierto intervalo de tiempo muy reducido, sin embargo las condiciones de la oficina pueden variar drásticamente a lo largo del año debido a cambios clímaticos y demás.

Por último, a continuación se unen los datasets y se determina que efectivamente el dataset es linealmente separable.

In [57]:
all_office_dataset=pd.concat([train,test1,test2])

In [58]:
all_x_office=all_office_dataset[["Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy","day","month","year","hour","minute"]]
all_y_office=all_office_dataset["Occupancy"]

In [59]:
isLinear(all_x_office,all_y_office)

Intercept: [-2.38286181]
Coefficients: [[-1.61743591e-02  4.72225656e-03  9.01349968e-04  3.11452847e-04
  -4.43754755e-05  4.14242710e+00 -4.25415246e-03  1.67865721e-13
   1.67347025e-10 -1.98473287e-03  6.44935145e-05]]
Los datos son linealmente separables.


Por último a continuación se muestran algunos ejemplos del funcionamiento del modelo, tomando algunos ejemplos aleatorios del dataset de test

In [76]:
sample_office=test2.sample()
feature_sample_office=sample_office[["Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy","day","month","year","hour","minute"]]
real_value_office=sample_office["Occupancy"]
prediction_value_office=office_ocupancy.predict(feature_sample_office.values)

print(f"el valor real es: {real_value_office.iloc[0]} \nel valor de la predicción es: {prediction_value_office[0].item()}")

el valor real es: 0 
el valor de la predicción es: 0
